In [58]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [70]:
stk_dreturn = pd.read_csv('10Y R3000.csv', parse_dates=['date'])
stk_dreturn = stk_dreturn.rename(columns={'date':'Date'})
volatility_df = stk_dreturn.pivot_table(index='Date', columns='TICKER', values='VOL', aggfunc='mean')
original_price = stk_dreturn.pivot_table(index='Date', columns='TICKER', values='PRC', aggfunc='mean')
r3000_dreturn = pd.read_excel('rusell member.xlsx')
r3000_dreturn['Ticker'] = r3000_dreturn['Ticker'].str.split().str[0]
stk_dreturn = stk_dreturn[['Date', 'TICKER', 'RETX']]
stk_dreturn = stk_dreturn.rename(columns = {'TICKER':'Ticker'})
stk_dreturn = pd.merge(left=stk_dreturn, right=r3000_dreturn, on='Ticker', how='left')
stk_dreturn = stk_dreturn.dropna(subset = ['GICS Sector'])
cs_dreturn = pd.read_excel('communication service.xlsx', parse_dates=['Date']).rename(columns={'% Change': 'RET'})
cd_dreturn = pd.read_excel('Consumer Discretionary.xlsx').rename(columns={'% Change': 'RET'})
CS_dreturn = pd.read_excel('Consumer Staples.xlsx').rename(columns={'% Change': 'RET'})
en_dreturn = pd.read_excel('energy.xlsx').rename(columns={'% Change': 'RET'})
fi_dreturn = pd.read_excel('financial.xlsx').rename(columns={'% Change': 'RET'})
hc_dreturn = pd.read_excel('health care.xlsx').rename(columns={'% Change': 'RET'})
in_dreturn = pd.read_excel('industrials.xlsx').rename(columns={'% Change': 'RET'})
it_dreturn = pd.read_excel('information technology.xlsx').rename(columns={'% Change': 'RET'})
ma_dreturn = pd.read_excel('material.xlsx').rename(columns={'% Change': 'RET'})
re_dreturn = pd.read_excel('real estate.xlsx').rename(columns={'% Change': 'RET'})
ut_dreturn = pd.read_excel('utility.xlsx').rename(columns={'% Change': 'RET'})
def add_sector_name_label(sector_data, name):
    sector_data = sector_data[['Date', 'RET']]
    sector_data['GICS_Sector'] = name  # Setting the sector name dynamically based on input
    
    return sector_data
cd_dreturn = add_sector_name_label(cd_dreturn, 'Consumer Discretionary')
cs_dreturn = add_sector_name_label(cs_dreturn, 'Communication Services')
CS_dreturn = add_sector_name_label(CS_dreturn, 'Consumer Staples')
en_dreturn = add_sector_name_label(en_dreturn, 'Energy')
fi_dreturn = add_sector_name_label(fi_dreturn, 'Financials')
hc_dreturn = add_sector_name_label(hc_dreturn, 'Health Care')
in_dreturn = add_sector_name_label(in_dreturn, 'Industrials')
it_dreturn = add_sector_name_label(it_dreturn, 'Information Technology')
ma_dreturn = add_sector_name_label(ma_dreturn, 'Materials')
re_dreturn = add_sector_name_label(re_dreturn, 'Real Estate')
ut_dreturn = add_sector_name_label(ut_dreturn, 'Utilities')
all_sectors = pd.concat([cs_dreturn, cd_dreturn, CS_dreturn, en_dreturn, hc_dreturn, in_dreturn, it_dreturn, ma_dreturn, re_dreturn, ut_dreturn])
all_sectors.columns=['Date','RET','GICS Sector']
result = pd.merge(left=stk_dreturn, right=all_sectors, on=['Date','GICS Sector'], how='left')
result = result.drop(columns = ['Price'])
result['RET_t-2'] = result['RET'].shift(2)
result['RET_t-1'] = result['RET'].shift(1)
result['RET_t+2'] = result['RET'].shift(-2)
result['RET_t+1'] = result['RET'].shift(-1)
result = result.iloc[2:-3]
result = result.dropna()
def regression_find_residual(group):
    X = group[['RET_t-2', 'RET_t-1', 'RET', 'RET_t+2', 'RET_t+1']]
    X = sm.add_constant(X)  
    y = group['RETX'] 
    
    X = X.dropna()
    y = y.loc[X.index]
    
    
    model = sm.OLS(y, X, missing='drop').fit()

    # Add residuals to the group DataFrame
    group['residual'] = model.resid.reindex(group.index).fillna(0) 
    
    return group
result[['RET_t-2', 'RET_t-1', 'RET', 'RET_t+2', 'RET_t+1', 'RETX']].dtypes
result['RETX'] = pd.to_numeric(result['RETX'], errors='coerce')
result['RETX'] = result['RETX'].dropna()
data = regression_find_residual(result)
data['residual'] = np.log1p(data['residual'])
residual_df = data.pivot_table(index='Date', columns='Ticker', values='residual', aggfunc='mean')

residual_df.drop('2014-01-02',axis=0,inplace=True)
residual_df.drop('2014-01-03',axis=0,inplace=True)
residual_df.dropna(axis=1,inplace = True, how='any')

volatility_df.drop('2014-01-02',axis=0,inplace=True)
volatility_df.drop('2014-01-03',axis=0,inplace=True)
volatility_df.drop('2023-12-27',axis=0,inplace=True)
volatility_df.drop('2023-12-28',axis=0,inplace=True)
volatility_df.drop('2023-12-29',axis=0,inplace=True)
volatility_df.dropna(axis=1,inplace = True, how='any')

original_price.drop('2014-01-02',axis=0,inplace=True)
original_price.drop('2014-01-03',axis=0,inplace=True)
original_price.drop('2023-12-27',axis=0,inplace=True)
original_price.drop('2023-12-28',axis=0,inplace=True)
original_price.drop('2023-12-29',axis=0,inplace=True)
original_price.dropna(axis=1,inplace = True, how='any')

/var/folders/24/vr3g0t6d6rq244s67kr49w780000gn/T/ipykernel_30138/652869122.py:1: DtypeWarning: Columns (8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  stk_dreturn = pd.read_csv('10Y R3000.csv', parse_dates=['date'])
/var/folders/24/vr3g0t6d6rq244s67kr49w780000gn/T/ipykernel_30138/652869122.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sector_data['GICS_Sector'] = name  # Setting the sector name dynamically based on input
/var/folders/24/vr3g0t6d6rq244s67kr49w780000gn/T/ipykernel_30138/652869122.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata

In [71]:
residual_df.head()

Ticker,A,AAL,AAOI,AAON,AAP,AAPL,AAT,ABBV,ABG,ABM,...,XRAY,XRX,XYL,YELP,YORW,YUM,ZBRA,ZEUS,ZTS,ZUMZ
Date,,,,,,,,,,,,,,,,,,,,,
2014-01-06,-0.001351,0.024474,0.001132,-0.009833,-0.003504,0.007948,-0.008472,-0.033506,0.000036,0.002792,...,-0.002452,0.010808,0.004561,0.054161,-0.007894,0.005263,-0.000833,-0.000090,0.001384,0.000334
2014-01-07,0.003633,-0.009891,0.019158,0.009176,0.006603,-0.016561,0.007123,-0.008715,-0.003829,-0.000983,...,0.004776,-0.000998,-0.002315,0.008001,0.030530,0.008287,0.001031,0.022838,-0.006934,0.017209
2014-01-08,0.007160,0.027391,-0.006382,0.005237,-0.006094,0.005388,-0.016987,-0.011821,-0.012006,-0.005165,...,-0.007354,-0.010009,0.000244,0.091870,-0.008043,0.001307,-0.001678,-0.021789,-0.020603,-0.014189
2014-01-09,-0.007509,0.059186,-0.029090,-0.009842,0.012271,-0.007364,-0.007604,0.009211,-0.003012,0.002196,...,-0.004138,0.002947,0.000247,0.019023,-0.018209,-0.018290,0.010021,0.016943,-0.000893,-0.077329
2014-01-10,0.004032,-0.004309,-0.013810,-0.006694,0.014404,-0.009129,0.009109,-0.011173,-0.012458,-0.004038,...,-0.004053,-0.007423,0.006146,0.043882,-0.006027,-0.004307,-0.000200,0.011202,0.013198,-0.056280


In [72]:
volatility_df.tail()

TICKER,A,AA,AAL,AAN,AAOI,AAON,AAP,AAPL,AAT,AB,...,XYL,YELP,YORW,YUM,Z,ZBRA,ZEUS,ZION,ZTS,ZUMZ
Date,,,,,,,,,,,,,,,,,,,,,
2023-12-19,1523339.0,7967243.0,22358797.0,315014.0,2971865.0,607087.0,1868869.0,40229887.0,522034.0,570652.0,...,1547027.0,822404.0,62118.0,1704157.0,5216125.0,463713.0,124538.0,1860011.0,1600281.0,219215.0
2023-12-20,2121066.0,7516945.0,26283849.0,301843.0,2833625.0,520232.0,1645991.0,51740699.0,385421.0,663833.0,...,1443053.0,1269852.0,67391.0,1321412.0,4978010.0,327529.0,130267.0,2052916.0,1833160.0,225980.0
2023-12-21,1685889.0,5566085.0,30018020.0,331102.0,3589333.0,321631.0,975654.0,46086400.0,340858.0,359843.0,...,1037042.0,922706.0,35916.0,1211633.0,7886035.0,279569.0,59031.0,1701553.0,2118571.0,128287.0
2023-12-22,1204144.0,9066769.0,24831320.0,311286.0,2202786.0,313246.0,1174732.0,36702455.0,307738.0,471038.0,...,829320.0,520353.0,29875.0,991533.0,2601730.0,222001.0,103655.0,1507910.0,1548412.0,160428.0
2023-12-26,948390.0,4505900.0,21708238.0,208507.0,2968734.0,201067.0,1072419.0,28541150.0,146419.0,332241.0,...,440259.0,339193.0,38964.0,627506.0,2806905.0,218146.0,63396.0,1122592.0,814641.0,122386.0


In [68]:
original_price.tail()

TICKER,A,AA,AAL,AAN,AAOI,AAON,AAP,AAPL,AAT,AB,...,XYL,YELP,YORW,YUM,Z,ZBRA,ZEUS,ZION,ZTS,ZUMZ
Date,,,,,,,,,,,,,,,,,,,,,
2023-12-19,139.78999,32.42,14.30,11.19,23.01,72.36,60.86,196.94000,22.67,32.75,...,111.58,46.82,39.19,130.24001,57.85,269.42001,65.02,43.93,198.08,19.46
2023-12-20,138.17999,30.65,13.98,10.89,20.62,71.78,59.45,194.83000,22.63,31.62,...,109.67,47.60,38.43,129.03999,56.86,261.60999,64.34,42.54,194.63,18.85
2023-12-21,138.94000,32.02,14.35,11.17,20.74,72.39,60.69,194.67999,22.75,31.17,...,111.69,48.24,38.46,129.98000,58.75,265.31000,66.31,43.62,194.66,19.63
2023-12-22,139.57001,33.77,14.31,11.11,20.90,73.41,61.25,193.60001,22.65,31.78,...,112.64,48.43,38.67,130.00000,58.79,269.41000,67.10,44.00,194.98,19.68
2023-12-26,139.81000,33.87,14.11,11.19,22.82,74.15,60.92,193.05000,23.04,31.45,...,113.62,48.19,39.51,130.25999,58.68,275.50000,68.14,44.88,195.50,19.80
